In [1]:
# LIMS access:
import psycopg2
import psycopg2.extras
# import sciris as sc
import pandas as pd
from visual_behavior.data_access import loading
import visual_behavior.database as db

In [4]:
def get_paired_planes(session_id):
        ''' 
        Get paired experiments for given session. 
        This function will first query LIMS. (Query provided by Wayne)
        But since LIMS does not have this information for a lot of older mesoscope sesions, if the query returns nothing, 
        it will try and parce cell extraction input json to get paired planes
        :param ophys_session_id
        :return: list of two elememnt list, where each sublist is two experiment IDs of a pair of coupled planes. In order of acquisition. 
        '''
        pairs = []
        try:
            query = (f"""SELECT
            os.id as session_id,
            oe.id as exp_id,
            oe.ophys_imaging_plane_group_id as pair_id,
            oipg.group_order
            FROM ophys_sessions os
            JOIN ophys_experiments oe ON oe.ophys_session_id=os.id
            JOIN ophys_imaging_plane_groups oipg ON oipg.id=oe.ophys_imaging_plane_group_id
            WHERE os.id = {session_id}
            ORDER BY exp_id
            """)
            
            pairs_df = db.lims_query(query)
            
        except Exception as e:
            print("Unable to query LIMS database: {}".format(e))
        if len(pairs_df) > 0:
            num_groups = pairs_df['group_order'].drop_duplicates().values
            for i in num_groups:
                pair = [exp_id for exp_id in pairs_df.loc[pairs_df['group_order'] == i].exp_id]
                pairs.append(pair)
        else:
            print(f"Lims returned no group information about session {self.session_id}, using hardcoded splitting json filename")
            splitting_json = self.get_splitting_json()
            with open(splitting_json, "r") as f:
                data = json.load(f)
            for pg in data.get("plane_groups", []):
                pairs.append([p["experiment_id"] for p in pg.get("ophys_experiments", [])])
        return pairs
    
def refactor_sessions_table_mesoscope_for_qc():
    '''
    Refactor the sessions table for emoscope decrosstalking QC.
    :param 
    :return: pandas DataFrame, refactored table
    '''
    session_table = loading.get_filtered_ophys_session_table()
    meso_only_sessions = session_table.loc[session_table.equipment_name == 'MESO.1']
    meso_only_sessions_filtered = meso_only_sessions #.drop(columns=['ophys_experiment_id', 'at_least_one_experiment_passed', 'age_in_days', 'at_least_one_experiment_passed', 'behavior_session_id', 'donor_id', 'full_genotype', 'model_outputs_available', 'reporter_line', 'session_name', 'sex', 'specimen_id'])
    meso_table = pd.concat([meso_only_sessions_filtered, pd.DataFrame(columns=['ophys_experiment_ids, paired'])])
    for session in meso_table.index:
        paired_planes = get_paired_planes(session)
        meso_table.at[session,'ophys_experiment_ids, paired'] = paired_planes
    return meso_table


In [5]:
meso_table = refactor_sessions_table_mesoscope_for_qc()
meso_table['container_id'] = meso_table['container_id'].astype(int)
meso_table = meso_table.reset_index().rename(columns={'index':'ophys_session_id'})

columms_to_show = [
    'ophys_session_id',
    'ophys_experiment_ids, paired',
    'date_of_acquisition', 
    'driver_line',
    'equipment_name', 
    'mouse_id',
    'project_code', 
    'session_type',
]
meso_table[columms_to_show]

/home/dougo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:51: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



KeyError: "['mouse_id'] not in index"

In [13]:
meso_table.columns

Index(['container_id', 'container_workflow_state', 'date_of_acquisition',
       'driver_line', 'equipment_name', 'mouse_id',
       'ophys_experiment_ids, paired', 'project_code', 'session_type',
       'session_workflow_state'],
      dtype='object')

In [6]:
import os
def get_roi_overlap_plots_links(session_id, plots_dir = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/single_cell_plots/mesoscope_decrosstalk'):
    """
    function to build links ot the roi-level plots given session_id
    session_id : int, session ID form lims
    plots_dir: str, path to outer directory 
    returns dict, where {'pair_0_overlaps' : "path_to_roi_level_dir"}
    """
    session_path = os.path.join(plots_dir, f"session_{session_id}")
    roi_links = {}
    pairs = get_paired_planes(session_id)
    for i, pair in enumerate(pairs):
        pair_dir_path = os.path.join(session_path, f"pair_{i}_overlaps")
        if os.path.isdir(pair_dir_path):
            roi_links[f'pair_{i}'] = pair_dir_path
        else:
            roi_links[f'pair_{i}']  = "roi level plots don't exist"        
    return roi_links

In [9]:
link_list = [v for k,v in get_roi_overlap_plots_links(1048363441).items()]
link_list

['/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/single_cell_plots/mesoscope_decrosstalk/session_1048363441/pair_0_overlaps',
 '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/single_cell_plots/mesoscope_decrosstalk/session_1048363441/pair_1_overlaps',
 '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/single_cell_plots/mesoscope_decrosstalk/session_1048363441/pair_2_overlaps',
 '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/single_cell_plots/mesoscope_decrosstalk/session_1048363441/pair_3_overlaps']

In [10]:
get_roi_overlap_plots_links(866197765)

{'pair_0': "roi level plots don't exist",
 'pair_1': "roi level plots don't exist",
 'pair_2': "roi level plots don't exist",
 'pair_3': "roi level plots don't exist"}

In [11]:
functions.load_container_plot_options()

NameError: name 'functions' is not defined